In [1]:
# requires "ollama serve" to be running in another terminal

# pip install python-docx
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA

from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

In [11]:
from langchain_community.chat_models import ChatLlamaCpp
#from llama_cpp import Llama
local_model_path = "/Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instruct-GGUF/qwen2.5-coder-14b-instruct-q4_0.gguf"
# Load the Llama model for embeddings and chat
#llama_model = Llama(model_path=local_model_path, verbose=False, n_ctx=131000)
# Convert Llama to ChatLlamaCpp
llama_model = ChatLlamaCpp(
    model_path=local_model_path,
    temperature=0.1,
    n_ctx=2048,
    verbose=True
)

llama_model_loader: loaded meta data with 26 key-value pairs and 579 tensors from /Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instruct-GGUF/qwen2.5-coder-14b-instruct-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 Coder 14B Instruct AWQ
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-AWQ
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5-Coder
llama_model_loader: - kv   5:                         general.size_label str              = 14B
llama_model_loader: - kv   6:                          qw

In [3]:
embedding_model_dir = "/Users/maxhuang/projects/llm/models/all-MiniLM-L6-v2"
from langchain.embeddings import HuggingFaceEmbeddings

# Load HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_dir)

/var/folders/x2/zk253tmn003dnbcnhrtjggc80000gn/T/ipykernel_18489/1033093798.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_dir)
/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Create index (can be reused):
data_path = "/Users/maxhuang/projects/llm/langchain-book-examples/data"
loader = DirectoryLoader(data_path, glob="**/*.txt", show_progress=True)
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

persist_directory = 'cache'

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding_model,
    persist_directory=persist_directory)

vectorstore.persist()

# Try reloading index from disk and using for search:

persist_directory = 'cache'

vectorstore = Chroma(persist_directory=persist_directory,
                     embedding_function=embedding_model,
                    )

retriever = vectorstore.as_retriever()

# Initialize memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create conversational chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llama_model,
    retriever=retriever,
    memory=memory,
    verbose=True
)


100%|██████████| 4/4 [00:00<00:00, 49.16it/s]


In [15]:
# Example usage
while True:
    query = input("Ask a question: ")
    response = qa_chain(query)
    print(response['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What documents are in the dataset?
Assistant: I don't know.
Human: Who says that economics is bullshit?
Assistant: I don't know.
Follow Up Input: Who says that economics is bullshit?
Standalone question:


Llama.generate: 3 prefix-match hit, remaining 103 prompt tokens to eval

llama_print_timings:        load time =    7931.50 ms
llama_print_timings:      sample time =       3.02 ms /     8 runs   (    0.38 ms per token,  2651.64 tokens per second)
llama_print_timings: prompt eval time =    7768.15 ms /   103 tokens (   75.42 ms per token,    13.26 tokens per second)
llama_print_timings:        eval time =    1007.47 ms /     7 runs   (  143.92 ms per token,     6.95 tokens per second)
llama_print_timings:       total time =    8782.81 ms /   110 tokens
Llama.generate: 3 prefix-match hit, remaining 125 prompt tokens to eval



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Economics is the social science that studies the behavior of individuals, households, and

Economics is the social science that studies the behavior of individuals, households, and

Economics is the social science that studies the behavior of individuals, households, and

Economics is the social science that studies the behavior of individuals, households, and
Human: Who claims that economics is nonsense?



llama_print_timings:        load time =    7931.50 ms
llama_print_timings:      sample time =       1.96 ms /     6 runs   (    0.33 ms per token,  3065.92 tokens per second)
llama_print_timings: prompt eval time =    9469.48 ms /   125 tokens (   75.76 ms per token,    13.20 tokens per second)
llama_print_timings:        eval time =     688.79 ms /     5 runs   (  137.76 ms per token,     7.26 tokens per second)
llama_print_timings:       total time =   10164.05 ms /   130 tokens



> Finished chain.

> Finished chain.
I don't know.


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What documents are in the dataset?
Assistant: I don't know.
Human: Who says that economics is bullshit?
Assistant: I don't know.
Human: Who says that economics is bullshit?
Assistant: I don't know.
Follow Up Input: 
Standalone question:


Llama.generate: 3 prefix-match hit, remaining 113 prompt tokens to eval
 25%|██▌       | 1/4 [19:37<58:51, 1177.14s/it]


KeyboardInterrupt: 